<a href="https://colab.research.google.com/github/jpengtelecom/Estudo-de-Caso/blob/main/Estudo_de_Caso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install numpy
!pip install matplotlib

In [48]:
import numpy as np
import matplotlib.pyplot as plt

# Parâmetros do sistema
lambda1 = 200  # taxa média de chegada (pacotes/segundo)
mu1 = 5      # taxa média de atendimento mais lenta (pacotes/segundo)
mu2 = 5      # taxa média de atendimento mais rápida (pacotes/segundo)
J = 10          # tamanho do buffer de fila
m = 5           # número mínimo de elementos para comutação de taxa
N = 10000        # número de pacotes a serem simulados

# Inicialização das variáveis de simulação
t = 0           # tempo inicial
nq = 0          # número de elementos na fila
ns = 0          # número de elementos no servidor
k = 0           # número de pacotes que passaram pelo sistema
ta = np.random.exponential(1/lambda1)   # tempo da próxima chegada
td1 = float('inf')  # tempo da próxima partida no servidor 1
td2 = float('inf')  # tempo da próxima partida no servidor 2
tp = []         # lista de tempos de partidas completadas
ts = []         # lista de tempos no sistema para cada pacote
proxima_chegada = t + ta  # tempo de chegada do próximo pacote

# Simulação
while k < N:  # número máximo de pacotes a serem simulados
    if proxima_chegada <= np.minimum(td1, td2):  # próxima ação é uma chegada
        t = proxima_chegada
        if ns < 2:  # se há servidor disponível
            ns += 1
            if ns == m:  # comutação de taxa
                td1 = t + np.random.exponential(1/mu2)
            else:
                td1 = t + np.random.exponential(1/mu1)
        else:  # não há servidor disponível, colocar na fila
            if nq < J:
                nq += 1
        k += 1
        ta = np.random.exponential(1/lambda1)
        proxima_chegada = t + ta
    else:  # próxima ação é uma partida
        t = min(td1, td2)
        if t == td1:
            ns -= 1
            tp.append(t)
            ts.append(t - (t - ta))  # tempo no sistema é tempo atual - tempo de chegada do pacote
            if ns >= m:
                td1 = t + np.random.exponential(1/mu2)
            else:
                if nq > 0:
                    nq -= 1
                    td1 = t + np.random.exponential(1/mu1)
                else:
                    td1 = float('inf')
        else:
            ns -= 1
            tp.append(t)
            ts.append(t - (t - ta))  # tempo no sistema é tempo atual - tempo de chegada do pacote
            if ns >= m:
                td2 = t + np.random.exponential(1/mu2)
            else:
                if nq > 0:
                    nq -= 1
                    td2 = t + np.random.exponential(1/mu1)
                else:
                    td2 = float('inf')

# Calcular métricas adicionais
tempo_medio_sistema = np.mean(ts) if len(ts) > 0 else 0
tempo_medio_fila = np.mean(np.maximum((ts - (1/mu1)), 0))  # considera apenas o tempo em fila
num_medio_pacotes_sistema = tempo_medio_sistema * lambda1
num_medio_pacotes_fila = tempo_medio_fila * lambda1 * nq

# Resultados
print("Tempo médio no sistema:", tempo_medio_sistema)
print("Tempo médio de pacotes na fila:", tempo_medio_fila)
print("Número médio de pacotes no sistema:", num_medio_pacotes_sistema)
print("Número médio de pacotes na fila:", num_medio_pacotes_fila)


TypeError: unsupported operand type(s) for -: 'list' and 'float'